# Contents:
1. Read images

### Import relevant libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

### Read from Directory

In [2]:
image_size = 256
batch_size = 32

In [3]:
dataset = tf.keras.utils.image_dataset_from_directory(
    "potato_dataset",
    labels='inferred',
    label_mode = 'int',
    batch_size = batch_size,
    image_size = (image_size,image_size),
    shuffle = True,
    seed= 42
)

Found 3302 files belonging to 3 classes.


In [4]:
class_names = dataset.class_names
class_names

['early_blight', 'late_blight', 'normal']

### Check if the data is balanced

In [5]:
# Create a dictionary to store class counts
class_counts = {}

# Loop through the dataset and count elements in each class
for images, labels in dataset:
    for label in labels.numpy():
        if label not in class_counts:
            class_counts[label] = 0
        class_counts[label] += 1

# Print the counts for each class
for label, count in class_counts.items():
    print(f'Class {label}: {count}')

min_count = min(class_counts)

Class 1: 1149
Class 0: 1133
Class 2: 1020


The data is distributed in acceptable range

### Split the data

In [6]:
def split_data(dataset, train_split = 0.8, val_split = 0.1, shuffle = True, shuffle_size= 10000):
    train_count = int(len(dataset)*train_split)
    val_count = int(len(dataset)*val_split)
    
    if shuffle:
        dataset = dataset.shuffle(shuffle_size, seed = 42)

    train_dataset = dataset.take(train_count)
    val_test_dataset = dataset.skip(train_count)

    validation_dataset = val_test_dataset.take(val_count)
    test_dataset = val_test_dataset.skip(val_count)

    return train_dataset, validation_dataset, test_dataset

In [7]:
train, validation, test = split_data(dataset)
print(f"train size = {len(train)}")
print(f"validation size = {len(validation)}")
print(f"test size = {len(test)}")

train size = 83
validation size = 10
test size = 11


### Cache and prefetch the data

In [8]:
train = train.cache().shuffle(10000).prefetch(buffer_size = tf.data.AUTOTUNE)
validation = validation.cache().shuffle(10000).prefetch(buffer_size = tf.data.AUTOTUNE)
test = test.cache().shuffle(10000).prefetch(buffer_size = tf.data.AUTOTUNE)

### Produce layers for model

In [9]:
resize_rescale_layer = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(image_size,image_size),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [10]:
data_augmentation_layer = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.3),
    layers.RandomZoom(0.2)
])

### Model


In [11]:
input_shape = (batch_size, image_size,image_size, 3)

def conv2D_layer(filter):
    return tf.keras.layers.Conv2D(
        filters = filter, 
        kernel_size=(3,3),
        padding="valid", # no padding
        activation="relu",
        input_shape = input_shape
    )
    

model = tf.keras.Sequential([
    # preprocessing layers
    resize_rescale_layer,
    data_augmentation_layer,

    # Convolutional layer
    conv2D_layer(128),
    tf.keras.layers.MaxPool2D((2,2)),
    conv2D_layer(64),
    tf.keras.layers.MaxPool2D((2,2)),
    conv2D_layer(64),
    tf.keras.layers.MaxPool2D((2,2)),
    conv2D_layer(64),
    tf.keras.layers.MaxPool2D((2,2)),
    conv2D_layer(32),
    tf.keras.layers.MaxPool2D((2,2)),

    tf.keras.layers.Flatten(),

    # Dense layer
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(3, activation="softmax")
])

model.build(input_shape=input_shape)

In [12]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (32, 256, 256, 3)         0         
                                                                 
 sequential_1 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 128)     3584      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 128)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      73792     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                  

In [13]:
model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
    metrics = ['accuracy']
)

EPOCHS = 15

history = model.fit(
    train,
    epochs= EPOCHS,
    batch_size=batch_size,
    verbose =1,
    validation_data=validation
)

Epoch 1/15


In [ ]:
model.evaluate(test)

11/11 [==============================] - 10s 696ms/step - loss: 0.0814 - accuracy: 0.9716


[0.08135230094194412, 0.9715909361839294]

### Visualization of training

In [ ]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss= history.history['loss']
val_loss = history.history['val_loss']

NameError: name 'history' is not defined

In [ ]:
''' 
import matplotlib.pyplot as plt

# Assuming you have already stored the data in the variables as mentioned in your question

# Plot training & validation accuracy values
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(accuracy) + 1), accuracy, label='Training Accuracy')
plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(range(1, len(loss) + 1), loss, label='Training Loss')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

'''

### Save the model

In [ ]:
model_version = 1
model.save(f"../saved_models/{model_version}")